# Q1 利益冲突检测（供应商与员工）

目标：依据PDF示例提示，识别以下风险场景：
- 1.1 多个供应商共用银行账号（提示：清洗账号，去除非数字字符与掩码字符X/x；统一大小写）
- 1.2 多个供应商共用地址（提示：分别针对“供应商注册地址”与“通讯地址省份+城市”进行归一化；注意过于粗粒度的城市汇总可能产生较多误报）
- 1.3 供应商与员工共用联系方式（提示：手机/座机/邮箱与员工的电话/邮箱类字段统一清洗再匹配）

为避免运行超时，以下分析采用子集/列裁剪的方式进行，并提供可视化与文字说明。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)

case12_path = Path('/workspace/Sample Data.Case 1&2.xlsx')
vm = pd.read_excel(case12_path, sheet_name='Vendor Master')
em = pd.read_excel(case12_path, sheet_name='Employee Master')
print(vm.shape, em.shape)
vm.head(3), em.head(3)

## 1.1 多个供应商共用银行账号
方法：
- 清洗银行账号：去除非数字与字母X/x（保留掩码位），统一为大写；
- 以清洗后的账号分组，统计供应商数>1的情况；
- 可视化：Top10共享账号的供应商数量条形图。

In [ ]:
vm_bank = vm[['新系统供应商编码','新系统供应商名称','银行账号']].dropna(subset=['银行账号']).copy()
vm_bank['银行账号_clean'] = vm_bank['银行账号'].astype(str).str.replace(r'[^0-9Xx]','', regex=True).str.upper()
dup_bank = (vm_bank.groupby('银行账号_clean')
    .agg(vendors=('新系统供应商编码', lambda x: list(x)), names=('新系统供应商名称', lambda x: list(x)), n=('新系统供应商编码','nunique'))
    .reset_index())
dup_bank = dup_bank[dup_bank['n']>1].sort_values('n', ascending=False)
print('共享银行账号组数：', len(dup_bank))
dup_bank.head(10)

In [ ]:
plt.figure(figsize=(10,4))
sns.barplot(x='银行账号_clean', y='n', data=dup_bank.head(10), color='steelblue')
plt.xticks(rotation=45, ha='right')
plt.title('共享银行账号Top10的供应商数量')
plt.ylabel('供应商数量')
plt.xlabel('银行账号_clean')
plt.tight_layout()
plt.show()

## 1.2 多个供应商共用地址
方法：
- 注册地址：“供应商注册地址”分组统计；
- 通讯地址：将“通讯地址省份+通讯地址城市”作为通讯地址的归一化表示；
- 注意事项（PDF提示）：省市级别过粗可能产生较多共享现象，需要结合街道/门牌进一步核查以降低误报。

In [ ]:
# 注册地址共享
if '供应商注册地址' in vm.columns:
    tmp = vm[['新系统供应商编码','新系统供应商名称','供应商注册地址']].dropna().copy()
    tmp['供应商注册地址'] = tmp['供应商注册地址'].astype(str).str.strip()
    dup_addr_reg = (tmp.groupby('供应商注册地址')
        .agg(vendors=('新系统供应商编码', lambda x: list(x)), names=('新系统供应商名称', lambda x: list(x)), n=('新系统供应商编码','nunique'))
        .reset_index())
    dup_addr_reg = dup_addr_reg[dup_addr_reg['n']>1].sort_values('n', ascending=False)
    print('共享注册地址组数：', len(dup_addr_reg))
    display(dup_addr_reg.head(10))

# 通讯地址共享（省市拼接）
if {'通讯地址省份','通讯地址城市'}.issubset(vm.columns):
    vm['通讯地址'] = vm['通讯地址省份'].fillna('').astype(str) + vm['通讯地址城市'].fillna('').astype(str)
    tmp2 = vm[['新系统供应商编码','新系统供应商名称','通讯地址']].dropna().copy()
    tmp2['通讯地址'] = tmp2['通讯地址'].astype(str).str.strip()
    dup_addr_comm = (tmp2.groupby('通讯地址')
        .agg(vendors=('新系统供应商编码', lambda x: list(x)), names=('新系统供应商名称', lambda x: list(x)), n=('新系统供应商编码','nunique'))
        .reset_index())
    dup_addr_comm = dup_addr_comm[dup_addr_comm['n']>1].sort_values('n', ascending=False)
    print('共享通讯地址组数：', len(dup_addr_comm))
    display(dup_addr_comm.head(10))

    plt.figure(figsize=(10,4))
    sns.barplot(x='通讯地址', y='n', data=dup_addr_comm.head(10), color='coral')
    plt.xticks(rotation=45, ha='right')
    plt.title('共享通讯地址Top10的供应商数量（省市拼接）')
    plt.ylabel('供应商数量')
    plt.xlabel('通讯地址')
    plt.tight_layout()
    plt.show()

## 1.3 供应商与员工共用联系方式
方法：
- 供应商侧：业务联系人手机/座机/邮箱 → 统一小写、去空格；
- 员工侧：手机号码/私人邮箱等字段 → 统一清洗；
- 直接按联系字段内连接匹配，输出样例与数量，并可视化不同联系类型的匹配计数。

In [ ]:
em_cols = list(em.columns)
contact_cols_em = [c for c in em_cols if any(k in str(c) for k in ['电话','手机','座机','Email','邮箱'])]
print('员工表中疑似联系方式字段：', contact_cols_em)

vendor_contacts = pd.DataFrame()
for col in ['业务联系人手机','业务联系人座机','业务联系人邮箱']:
    if col in vm.columns:
        s = vm[['新系统供应商编码','新系统供应商名称', col]].rename(columns={col:'contact'}).dropna(subset=['contact']).copy()
        s['contact'] = s['contact'].astype(str).str.strip().str.lower()
        s['source'] = col
        vendor_contacts = pd.concat([vendor_contacts, s], ignore_index=True)

emp_contacts = pd.DataFrame()
emp_id_col = '员工编号' if '员工编号' in em.columns else em.columns[0]
emp_name_col = '姓名' if '姓名' in em.columns else (em.columns[1] if len(em.columns)>1 else 'name')
for col in contact_cols_em:
    s = em[[emp_id_col, emp_name_col, col]].copy()
    s = s.rename(columns={emp_id_col:'EmployeeID', emp_name_col:'EmployeeName', col:'contact'})
    s = s.dropna(subset=['contact'])
    s['contact'] = s['contact'].astype(str).str.strip().str.lower()
    s['source'] = col
    emp_contacts = pd.concat([emp_contacts, s], ignore_index=True)

shared = vendor_contacts.merge(emp_contacts, on='contact', how='inner')
print('供应商-员工共用联系方式匹配条数：', shared.shape[0])
shared.head(10)

plt.figure(figsize=(8,4))
sns.countplot(y='source', data=shared)
plt.title('匹配发生于不同联系方式类型的计数')
plt.tight_layout()
plt.show()

### 结果与审计建议
- 多供应商共用银行账号：建议穿透核查开户行证明、合同与付款记录，重点关注同账号关联公司是否存在关联关系或异常。
- 多供应商共用地址：省市层面的共享较多，需结合更细地址要素（街道/门牌）以及工商登记信息，过滤误报。
- 供应商与员工共用联系方式：匹配到的样例需结合人事记录、供应商背景调查确认是否存在关联交易或内外部人员勾连。

### 误报控制（基于PDF示例提示的补充）
- 银行账号含掩码字符X/x时，需结合原始凭证进一步确认；
- 地址共享若仅停留在省市维度，属于粗粒度共性，建议进一步分层校验；
- 联系方式匹配需排除公共邮箱（如info@、service@）与总机号码等公共渠道。

## 附加：更细粒度分析与结果导出

In [ ]:
from pathlib import Path
outputs_path = Path('/workspace/KPMG_HW1/outputs')
outputs_path.mkdir(parents=True, exist_ok=True)

# 导出共享账号/地址/联系方式清单
if 'dup_bank' in globals():
    dup_bank.to_csv(outputs_path/'Q1_shared_bank_accounts.csv', index=False)
if 'dup_addr_reg' in globals():
    dup_addr_reg.to_csv(outputs_path/'Q1_shared_registered_addresses.csv', index=False)
if 'dup_addr_comm' in globals():
    dup_addr_comm.to_csv(outputs_path/'Q1_shared_comm_addresses_city.csv', index=False)
if 'shared' in globals():
    shared.to_csv(outputs_path/'Q1_vendor_employee_shared_contacts.csv', index=False)

# 更细粒度：供应商名称归一化（去常见后缀）并统计
def normalize_name(s):
    s = str(s)
    for suf in ['有限公司','公司','商行','中心','超市','修部','营部','金店','水站','总汇','货店','艺店']:
        s = s.replace(suf,'')
    return s.strip()

vm['新系统供应商名称_norm'] = vm['新系统供应商名称'].apply(normalize_name)
name_counts = vm.groupby('新系统供应商名称_norm')['新系统供应商编码'].nunique().reset_index(name='供应商数')
name_counts = name_counts.sort_values('供应商数', ascending=False)
name_counts.head(10)
name_counts.to_csv(outputs_path/'Q1_vendor_name_normalization_counts.csv', index=False)

# 更细粒度：共享通讯地址的分布（省/市聚合）
if {'通讯地址省份','通讯地址城市'}.issubset(vm.columns):
    comm_city_counts = vm.groupby(['通讯地址省份','通讯地址城市'])['新系统供应商编码'].nunique().reset_index(name='供应商数')
    comm_city_counts = comm_city_counts.sort_values('供应商数', ascending=False)
    comm_city_counts.head(20)
    comm_city_counts.to_csv(outputs_path/'Q1_comm_address_city_counts.csv', index=False)

# 更细粒度：联系方式类型匹配分布
if 'shared' in globals():
    type_counts = shared.groupby('source').size().reset_index(name='count').sort_values('count', ascending=False)
    type_counts.to_csv(outputs_path/'Q1_shared_contact_type_counts.csv', index=False)
    type_counts.head(10)
